In [2]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
import csv
import pymongo
import time
import json

In [3]:
sleep_time = 3

In [4]:
#create lists of players and team columns for scraping data from basketball-reference.com
players_columns = ['player','Age','G','GS','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%',
           'eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','place','team','team_abbr']
team_columns = ['team','G','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%',
           'FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','wins','losses','place','lat','lon','city','team_abbr']


In [7]:
!which chromedriver
#for Mac
#executable_path = {'executable_path': '/usr/local/bin/chromedriver'}

#for Windows
executable_path = {'executable_path': 'chromedriver.exe'}

browser = Browser('chrome', **executable_path, headless=False)

'which' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
players_ds = []
teams_ds = []

#import teams.csv to combine teams information (place/city, team name, and team abbreviation) with players data
with open('teams.csv', mode='r') as csv_file:
    csv_data = csv.DictReader(csv_file)

#create a loop to scrape players and teams status data from basketball-reference.com
    for team in csv_data:
   
        url = "https://www.basketball-reference.com/teams/"+team['abbreviation']+"/2020.html"
        browser.visit(url)
        time.sleep(sleep_time)

        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        players_table = soup.find('table', id="totals")
        players_tbody = players_table.find("tbody")
        players_trows = players_tbody.find_all("tr")

        #scrape players status data from basketball-reference.com
        row = {}

        for tr in players_trows:
            i = 0
            td = tr.find_all("td")
            for column in players_columns:
                if column == 'place':
                    row[column] = team['place']
                elif column == 'team':
                    row[column] = team['team']
                elif column == 'team_abbr':
                    row[column] = team['abbreviation']
                else:
                    row[column] = td[i].text
                    i += 1
            players_ds.append(dict(row))

        team_table = soup.find('table', id='team_and_opponent')
        team_tbody = team_table.find('tbody')
        team_trow = team_tbody.find('tr')
        team_td = team_trow.find_all("td")
        team_misc = soup.find('table', id='team_misc').find('tbody').find('tr').find_all('td')

        #scrape players status data from basketball-reference.com
        row = {}
        i = 0
        
        for column in team_columns:
            if column == 'place':
                row[column] = team['place']
            elif column == 'team':
                row[column] = team['team']
            elif column == 'lat':
                row[column] = team['lat']
            elif column == 'lon':
                row[column] = team['lon']
            elif column == 'city':
                row[column] =team['city']
            elif column == 'wins':
                row[column] = team_misc[0].text
            elif column == 'losses':
                row[column] = team_misc[1].text
            elif column == 'team_abbr':
                row[column] = team['abbreviation']
            else:
                row[column] = team_td[i].text
                i += 1

        teams_ds.append(row)
    browser.quit()   

In [9]:
len(players_ds)

570

In [10]:
len(teams_ds)

30

In [11]:
players_ds

[{'player': 'Kyle Lowry',
  'Age': '33',
  'G': '52',
  'GS': '52',
  'MP': '1901',
  'FG': '309',
  'FGA': '741',
  'FG%': '.417',
  '3P': '153',
  '3PA': '432',
  '3P%': '.354',
  '2P': '156',
  '2PA': '309',
  '2P%': '.505',
  'eFG%': '.520',
  'FT': '255',
  'FTA': '296',
  'FT%': '.861',
  'ORB': '30',
  'DRB': '221',
  'TRB': '251',
  'AST': '399',
  'STL': '70',
  'BLK': '22',
  'TOV': '154',
  'PF': '168',
  'PTS': '1026',
  'place': 'Toronto',
  'team': 'Raptors',
  'team_abbr': 'TOR'},
 {'player': 'OG Anunoby',
  'Age': '22',
  'G': '63',
  'GS': '62',
  'MP': '1897',
  'FG': '267',
  'FGA': '527',
  'FG%': '.507',
  '3P': '82',
  '3PA': '215',
  '3P%': '.381',
  '2P': '185',
  '2PA': '312',
  '2P%': '.593',
  'eFG%': '.584',
  'FT': '59',
  'FTA': '86',
  'FT%': '.686',
  'ORB': '74',
  'DRB': '267',
  'TRB': '341',
  'AST': '99',
  'STL': '88',
  'BLK': '42',
  'TOV': '75',
  'PF': '155',
  'PTS': '675',
  'place': 'Toronto',
  'team': 'Raptors',
  'team_abbr': 'TOR'},
 {'p

In [12]:
teams_ds

[{'team': 'Raptors',
  'G': '64',
  'MP': '15460',
  'FG': '2596',
  'FGA': '5665',
  'FG%': '.458',
  '3P': '880',
  '3PA': '2369',
  '3P%': '.371',
  '2P': '1716',
  '2PA': '3296',
  '2P%': '.521',
  'FT': '1159',
  'FTA': '1448',
  'FT%': '.800',
  'ORB': '618',
  'DRB': '2275',
  'TRB': '2893',
  'AST': '1624',
  'STL': '565',
  'BLK': '316',
  'TOV': '924',
  'PF': '1376',
  'PTS': '7231',
  'wins': '46',
  'losses': '18',
  'place': 'Toronto',
  'lat': '43.6435',
  'lon': '-79.3791',
  'city': 'Toronto',
  'team_abbr': 'TOR'},
 {'team': 'Celtics',
  'G': '64',
  'MP': '15485',
  'FG': '2634',
  'FGA': '5735',
  'FG%': '.459',
  '3P': '794',
  '3PA': '2190',
  '3P%': '.363',
  '2P': '1840',
  '2PA': '3545',
  '2P%': '.519',
  'FT': '1170',
  'FTA': '1461',
  'FT%': '.801',
  'ORB': '683',
  'DRB': '2259',
  'TRB': '2942',
  'AST': '1459',
  'STL': '529',
  'BLK': '358',
  'TOV': '873',
  'PF': '1368',
  'PTS': '7232',
  'wins': '43',
  'losses': '21',
  'place': 'Boston',
  'lat':

In [13]:
players_df = pd.DataFrame(players_ds)
players_df

,2P,2P%,2PA,3P,3P%,3PA,AST,Age,BLK,DRB,...,PF,PTS,STL,TOV,TRB,eFG%,place,player,team,team_abbr
0,156,.505,309,153,.354,432,399,33,22,221,...,168,1026,70,154,251,.520,Toronto,Kyle Lowry,Raptors,TOR
1,185,.593,312,82,.381,215,99,22,42,267,...,155,675,88,75,341,.584,Toronto,OG Anunoby,Raptors,TOR
2,344,.506,680,115,.359,320,191,25,49,342,...,151,1253,53,132,399,.517,Toronto,Pascal Siakam,Raptors,TOR
3,156,.430,363,131,.388,338,317,25,13,162,...,118,845,91,110,180,.503,Toronto,Fred VanVleet,Raptors,TOR
4,257,.562,457,66,.398,166,73,30,42,310,...,138,801,24,100,414,.571,Toronto,Serge Ibaka,Raptors,TOR
5,165,.589,280,94,.398,236,78,26,17,138,...,93,721,55,64,161,.593,Toronto,Norman Powell,Raptors,TOR
6,93,.550,169,88,.396,222,107,22,12,171,...,104,495,32,65,217,.575,Toronto,Terence Davis,Raptors,TOR
7,137,.493,278,2,.118,17,90,25,22,152,...,105,375,44,49,254,.475,Toronto,Rondae Hollis-Jefferson,Raptors,TOR
8,48,.440,109,51,.402,127,124,35,33,199,...,101,275,28,43,228,.528,Toronto,Marc Gasol,Raptors,TOR
9,44,.484,91,23,.324,71,79,24,4,67,...,60,170,39,29,86,.485,Toronto,Patrick McCaw,Raptors,TOR


In [14]:
teams_df = pd.DataFrame(teams_ds)
teams_df

,2P,2P%,2PA,3P,3P%,3PA,AST,BLK,DRB,FG,...,TOV,TRB,city,lat,lon,losses,place,team,team_abbr,wins
0,1716,.521,3296,880,.371,2369,1624,316,2275,2596,...,924,2893,Toronto,43.6435,-79.3791,18,Toronto,Raptors,TOR,46
1,1840,.519,3545,794,.363,2190,1459,358,2259,2634,...,873,2942,Boston,42.3662,-71.0621,21,Boston,Celtics,BOS,43
2,1909,.523,3649,742,.362,2052,1682,349,2282,2651,...,923,2955,Philadelphia,39.9012,-75.172,26,Philadelphia,76ers,PHI,39
3,1734,.519,3340,824,.340,2423,1539,292,2409,2558,...,993,3103,Brooklyn,40.6826,-73.9754,34,Brooklyn,Nets,BRK,30
4,2007,.499,4024,631,.337,1872,1456,313,2274,2638,...,946,3069,New York,40.7505,-73.9934,45,New York,Knicks,NYK,21
5,1937,.567,3418,893,.356,2510,1685,390,2745,2830,...,969,3362,Milwaukee,43.0451,-87.9174,12,Milwaukee,Bucks,MIL,53
6,2092,.529,3957,648,.363,1787,1686,334,2211,2740,...,853,2784,Indianapolis,39.764,-86.1555,26,Indiana,Pacers,IND,39
7,1780,.511,3480,793,.348,2282,1510,265,2041,2573,...,1005,2721,Chicago,41.8807,-87.6742,43,Chicago,Bulls,CHI,22
8,1804,.515,3501,791,.367,2157,1589,299,2109,2595,...,1008,2753,Detroit,42.3411,-83.0553,46,Detroit,Pistons,DET,20
9,1892,.519,3645,727,.351,2070,1500,211,2172,2619,...,1073,2876,Cleveland,41.4965,-81.6882,46,Cleveland,Cavaliers,CLE,19


In [15]:
#rename columns of players dataframe
players_df = players_df.rename(columns={'2P':'Two_P','2P%':'Two_PP','2PA':'Two_PA','3P':'Three_P','3P%':'Three_PP','3PA':'Three_PA','FG%':'FGP','eFG%':'eFGP','FT%':'FTP'})
players_df.columns

Index(['Two_P', 'Two_PP', 'Two_PA', 'Three_P', 'Three_PP', 'Three_PA', 'AST',
       'Age', 'BLK', 'DRB', 'FG', 'FGP', 'FGA', 'FT', 'FTP', 'FTA', 'G', 'GS',
       'MP', 'ORB', 'PF', 'PTS', 'STL', 'TOV', 'TRB', 'eFGP', 'place',
       'player', 'team', 'team_abbr'],
      dtype='object')

In [16]:
#rename columns of teams dataframe
teams_df = teams_df.rename(columns={'2P':'Two_P','2P%':'Two_PP','2PA':'Two_PA','3P':'Three_P','3P%':'Three_PP','3PA':'Three_PA','FG%':'FGP','eFG%':'eFGP','FT%':'FTP'})
teams_df.columns

Index(['Two_P', 'Two_PP', 'Two_PA', 'Three_P', 'Three_PP', 'Three_PA', 'AST',
       'BLK', 'DRB', 'FG', 'FGP', 'FGA', 'FT', 'FTP', 'FTA', 'G', 'MP', 'ORB',
       'PF', 'PTS', 'STL', 'TOV', 'TRB', 'city', 'lat', 'lon', 'losses',
       'place', 'team', 'team_abbr', 'wins'],
      dtype='object')

In [17]:
#convert statuses of teams from strings to numbers
teams_df = teams_df.astype({'AST':int, 'BLK':int, 'DRB':int, 'FG':int, 'FGA':int, 'FGP':float, 'FT':int, 'FTA':int, 'FTP':float, 
                            'G':int, 'MP':int,'ORB':int, 'PF':int, 'PTS':int, 'STL':int, 'TOV':int, 'TRB':int, 'Three_P':int, 
                            'Three_PA':int,'Three_PP':float, 'Two_P':int, 'Two_PA':int, 'Two_PP':float,'losses':int,'wins':int})

In [18]:
teams_df['rk_PTS'] = teams_df['PTS'].rank(method='min',ascending=False)
teams_df['rk_AST'] = teams_df['AST'].rank(method='min',ascending=False)
teams_df['rk_STL'] = teams_df['STL'].rank(method='min',ascending=False)
teams_df['rk_BLK'] = teams_df['BLK'].rank(method='min',ascending=False)
teams_df['rk_TRB'] = teams_df['TRB'].rank(method='min',ascending=False)
teams_df['rk_TOV'] = teams_df['TOV'].rank(method='min',ascending=True)
teams_df['rk_Two_PP'] = teams_df['Two_PP'].rank(method='min',ascending=False)
teams_df['rk_Three_PP'] = teams_df['Three_PP'].rank(method='min',ascending=False)
teams_df['rk_FGP'] = teams_df['FGP'].rank(method='min',ascending=False)

teams_df

,Two_P,Two_PP,Two_PA,Three_P,Three_PP,Three_PA,AST,BLK,DRB,FG,...,wins,rk_PTS,rk_AST,rk_STL,rk_BLK,rk_TRB,rk_TOV,rk_Two_PP,rk_Three_PP,rk_FGP
0,1716,0.521,3296,880,0.371,2369,1624,316,2275,2596,...,46,14.0,12.0,2.0,17.0,15.0,12.0,17.0,6.0,20.0
1,1840,0.519,3545,794,0.363,2190,1459,358,2259,2634,...,43,13.0,25.0,9.0,7.0,10.0,7.0,19.0,12.0,17.0
2,1909,0.523,3649,742,0.362,2052,1682,349,2282,2651,...,39,18.0,8.0,8.0,8.0,9.0,11.0,14.0,14.0,10.0
3,1734,0.519,3340,824,0.340,2423,1539,292,2409,2558,...,30,21.0,19.0,28.0,22.0,3.0,25.0,19.0,26.0,26.0
4,2007,0.499,4024,631,0.337,1872,1456,313,2274,2638,...,21,24.0,26.0,15.0,18.0,4.0,14.0,27.0,27.0,24.0
5,1937,0.567,3418,893,0.356,2510,1685,390,2745,2830,...,53,2.0,7.0,20.0,3.0,1.0,20.0,1.0,16.0,2.0
6,2092,0.529,3957,648,0.363,1787,1686,334,2211,2740,...,39,19.0,6.0,22.0,11.0,24.0,3.0,9.0,12.0,2.0
7,1780,0.511,3480,793,0.348,2282,1510,265,2041,2573,...,22,27.0,22.0,1.0,27.0,28.0,26.0,25.0,23.0,24.0
8,1804,0.515,3501,791,0.367,2157,1589,299,2109,2595,...,20,23.0,15.0,16.0,20.0,26.0,27.0,23.0,9.0,17.0
9,1892,0.519,3645,727,0.351,2070,1500,211,2172,2619,...,19,26.0,23.0,25.0,30.0,18.0,29.0,19.0,22.0,20.0


In [19]:
#replace nane with 0
players_df.replace('', 0, inplace=True)
#convert statuses of teams from strings to numbers
players_df = players_df.astype({'AST':int, 'BLK':int, 'DRB':int, 'FG':int, 'FGA':int, 'FGP':float, 'FT':int, 'FTA':int, 'FTP':float, 
                            'G':int, 'GS': int, 'MP':int,'ORB':int, 'PF':int, 'PTS':int, 'STL':int, 'TOV':int, 'TRB':int, 'Three_P':int, 
                            'Three_PA':int,'Three_PP':float, 'Two_P':int, 'Two_PA':int, 'Two_PP':float, 'eFGP':float,'Age': int})

In [21]:
#conver teams dataframe to json format
teams_ds = teams_df.to_dict('records')
teams_ds

[{'Two_P': 1716,
  'Two_PP': 0.521,
  'Two_PA': 3296,
  'Three_P': 880,
  'Three_PP': 0.371,
  'Three_PA': 2369,
  'AST': 1624,
  'BLK': 316,
  'DRB': 2275,
  'FG': 2596,
  'FGP': 0.458,
  'FGA': 5665,
  'FT': 1159,
  'FTP': 0.8,
  'FTA': 1448,
  'G': 64,
  'MP': 15460,
  'ORB': 618,
  'PF': 1376,
  'PTS': 7231,
  'STL': 565,
  'TOV': 924,
  'TRB': 2893,
  'city': 'Toronto',
  'lat': '43.6435',
  'lon': '-79.3791',
  'losses': 18,
  'place': 'Toronto',
  'team': 'Raptors',
  'team_abbr': 'TOR',
  'wins': 46,
  'rk_PTS': 14.0,
  'rk_AST': 12.0,
  'rk_STL': 2.0,
  'rk_BLK': 17.0,
  'rk_TRB': 15.0,
  'rk_TOV': 12.0,
  'rk_Two_PP': 17.0,
  'rk_Three_PP': 6.0,
  'rk_FGP': 20.0},
 {'Two_P': 1840,
  'Two_PP': 0.519,
  'Two_PA': 3545,
  'Three_P': 794,
  'Three_PP': 0.363,
  'Three_PA': 2190,
  'AST': 1459,
  'BLK': 358,
  'DRB': 2259,
  'FG': 2634,
  'FGP': 0.459,
  'FGA': 5735,
  'FT': 1170,
  'FTP': 0.801,
  'FTA': 1461,
  'G': 64,
  'MP': 15485,
  'ORB': 683,
  'PF': 1368,
  'PTS': 7232,
 

In [22]:
#conver players dataframe to json format
players_ds = players_df.to_dict('records')
players_ds

[{'Two_P': 156,
  'Two_PP': 0.505,
  'Two_PA': 309,
  'Three_P': 153,
  'Three_PP': 0.354,
  'Three_PA': 432,
  'AST': 399,
  'Age': 33,
  'BLK': 22,
  'DRB': 221,
  'FG': 309,
  'FGP': 0.417,
  'FGA': 741,
  'FT': 255,
  'FTP': 0.861,
  'FTA': 296,
  'G': 52,
  'GS': 52,
  'MP': 1901,
  'ORB': 30,
  'PF': 168,
  'PTS': 1026,
  'STL': 70,
  'TOV': 154,
  'TRB': 251,
  'eFGP': 0.52,
  'place': 'Toronto',
  'player': 'Kyle Lowry',
  'team': 'Raptors',
  'team_abbr': 'TOR'},
 {'Two_P': 185,
  'Two_PP': 0.593,
  'Two_PA': 312,
  'Three_P': 82,
  'Three_PP': 0.381,
  'Three_PA': 215,
  'AST': 99,
  'Age': 22,
  'BLK': 42,
  'DRB': 267,
  'FG': 267,
  'FGP': 0.507,
  'FGA': 527,
  'FT': 59,
  'FTP': 0.686,
  'FTA': 86,
  'G': 63,
  'GS': 62,
  'MP': 1897,
  'ORB': 74,
  'PF': 155,
  'PTS': 675,
  'STL': 88,
  'TOV': 75,
  'TRB': 341,
  'eFGP': 0.584,
  'place': 'Toronto',
  'player': 'OG Anunoby',
  'team': 'Raptors',
  'team_abbr': 'TOR'},
 {'Two_P': 344,
  'Two_PP': 0.506,
  'Two_PA': 680,

In [23]:
#save data as json files
with open('nba_player_sdata.json', 'w') as f:
    json.dump(players_ds, f)

with open('nba_teams_data.json', 'w') as f:
    json.dump(teams_ds, f)